In [5]:
!pip install pymongo

In [10]:
import pymongo

from pymongo import MongoClient

In [11]:
client = MongoClient("localhost", 27017)

In [15]:
db = client['test']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [17]:
!pip install Faker

In [20]:
from faker import Faker
fake = Faker()

for _ in range(10):
    
  print(fake.name())

Joseph Lozano
Barbara Jordan
April Cunningham
Kim Vargas
Debra Hoover
Terry Woods
Debra Mcguire
Jennifer Lopez
Lori Alvarado
Christina Harris
